In [0]:
import pandas as pd
import numpy as np

invoices_copy = spark.read.table("workspace.default.invoices_silver")
#display(invoices_copy) Spark DataFrame

#invoices_copy['Invoice_friendly_ID'] = to nie zadziała bo PySpark DataFrame jest niemutowalny!!!

invoices_copy_temp = invoices_copy.toPandas() #Pandas DataFrame

invoices_copy_temp['invoice_friendly_id'] = invoices_copy_temp['invoice_id'].str.replace('INV-', '000INV')
invoices_copy = spark.createDataFrame(invoices_copy_temp)
display(invoices_copy)


In [0]:
invoices_copy.write.saveAsTable("workspace.default.invoices_copy_tests", mode="overwrite") #overwrite pozwoli Ci wywoływać to polecenie wiele razy


In [0]:
%sql
DESCRIBE HISTORY workspace.default.invoices_copy_tests

In [0]:
invoices_copy_temp['data_wiedzy'] = pd.Timestamp.now()
invoices_copy = spark.createDataFrame(invoices_copy_temp)
invoices_copy.display()
#zapis do Delta Table
#invoices_copy.write.saveAsTable("workspace.default.invoices_copy_tests", mode="overwrite") --> to wydupca exception, nie zgadza się schema, wskazuje, ze merge powinien pomóc

invoices_copy.write.mode("overwrite").option("mergeSchema", "true").saveAsTable("workspace.default.invoices_copy_tests")

In [0]:
%sql
DESCRIBE HISTORY workspace.default.invoices_copy_tests

In [0]:
%sql
SELECT *
FROM workspace.default.invoices_copy_tests VERSION AS OF 2

Teraz, skoro mamy 3 rózne wersje dla tej tabeli (z czego 0 i 1 są takie same), to spróbujemy przywrócić wersję 1

In [0]:
%sql
RESTORE TABLE workspace.default.invoices_copy_tests TO VERSION AS OF 1

In [0]:
%sql
SELECT * FROM workspace.default.invoices_copy_tests

In [0]:
%sql
DESCRIBE HISTORY workspace.default.invoices_copy_tests

Ale spoko, bo powrót do poprzedniej wersji wpada jako kolejny wpis

In [0]:
%sql
RESTORE TABLE workspace.default.invoices_copy_tests TO VERSION AS OF 2

In [0]:
%sql
SELECT *
FROM workspace.default.invoices_copy_tests